<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-functions,-packages-and-data" data-toc-modified-id="Load-functions,-packages-and-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load functions, packages and data</a></span></li><li><span><a href="#Regress-forward-beta" data-toc-modified-id="Regress-forward-beta-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Regress forward beta</a></span></li><li><span><a href="#Tabel----Panel-B" data-toc-modified-id="Tabel----Panel-B-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Tabel  - Panel B</a></span></li><li><span><a href="#Piecewise-regression---Tier-1-ratio" data-toc-modified-id="Piecewise-regression---Tier-1-ratio-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Piecewise regression - Tier 1 ratio</a></span></li></ul></div>

## Load functions, packages and data

In [1]:
"Beta function for rolling forward beta"

def market_beta_for(X,Y,min_win,max_win):
    """ 
    X = The independent variable which is the Market
    Y = The dependent variable which is the Stock
    min_win = minimum length of the window
    max_win = maximum length of the window
    
    It returns the alphas and the betas of
    the rolling regression
    """
    
    # all the observations
    obs = len(X)
    
    # initiate the betas with null values
    betas = np.full(obs, np.nan)
    
    # initiate the alphas with null values
    alphas = np.full(obs, np.nan)
    
    
    regressor = LinearRegression()
    for i in range(0,obs-min_win,1):
        if i >= obs-max_win: 
            if np.isnan(Y.to_numpy()[i+1 : -1].astype('float')).sum() > 0:
                pass
            else:
                regressor.fit(X.to_numpy()[i+1 : -1].reshape(-1,1), Y.to_numpy()[i+1: -1])
                betas[i]  = regressor.coef_[0]
                alphas[i]  = regressor.intercept_
        else: #the rolling window when periode = max window
            if np.isnan(Y.to_numpy()[i+1 : i+max_win].astype('float')).sum() > 0:
                pass
            else:
                regressor.fit(X.to_numpy()[i+1 : i+max_win].reshape(-1,1), Y.to_numpy()[i+1 : i+max_win])
                betas[i]  = regressor.coef_[0]
                alphas[i]  = regressor.intercept_
        
    return pd.DataFrame({'Alpha': alphas, 'Beta': betas})
 

In [2]:
%load_ext autoreload
%autoreload 2

"""
Load relevant packages 
"""
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import pandas_datareader as pdr
import datetime
from sklearn.linear_model import LinearRegression
import seaborn as sns
import pickle
from scipy.stats import kurtosis
from scipy.stats import skew
from typing import Union, List
import math as m
from sklearn import datasets, linear_model
import statsmodels.api as sm

In [3]:
"""
Set path to get save data
"""
path = "/Users/julielaustsen/Desktop/OneDrive/UNI/Speciale /Data/Eikon"
os.chdir(path) 

In [4]:
"""
Load saved dataframes and dictionareis
"""
    
with open('df_mkt', 'rb') as data_market_df:
    df_mkt = pickle.load(data_market_df)
    
with open('T1R_val', 'rb') as T1R_val_dict:
    T1R_val = pickle.load(T1R_val_dict)

with open('BER_val', 'rb') as BER_val_dict:
    BER_val = pickle.load(BER_val_dict) 

with open('T1RWA_val', 'rb') as T1RWA_val_dict:
    T1RWA_val = pickle.load(T1RWA_val_dict) 

In [5]:
T1R_val[0] #check data

,ew_excess_ret,ew_Tier_1_ratio,vw_excess_ret,vw_Tier_1_ratio
0,0.0,0.011625,0.0,0.018307
1,-0.037306,0.011625,-0.004638,0.018485
2,-0.049984,0.011625,-0.013725,0.018738
3,0.044883,0.011625,0.045492,0.018494
4,0.058086,0.011625,0.057528,0.0186
...,...,...,...,...
271,0.061387,0.04117,0.066489,0.041579
272,-0.007095,0.04117,0.020128,0.041579
273,0.053061,0.04117,0.055556,0.041579
274,-0.015688,0.04117,-0.030082,0.041578


## Regress forward beta

In [6]:
"""
Regres forward beta values for the teen portfolios sorted after Tier 1 ratio
"""

regress_calc_T1R = {}

for i in T1R_val.keys():
    regress_calc_T1R[i] = pd.DataFrame()
    regress_calc_T1R[i]['ew_forward_beta'] = market_beta_for(df_mkt['EUROPE IM (excess)'], T1R_val[i]['ew_excess_ret'],12, 24)['Beta']
    regress_calc_T1R[i]['vw_forward_beta'] = market_beta_for(df_mkt['EUROPE IM (excess)'], T1R_val[i]['vw_excess_ret'],12, 24)['Beta']                               
    regress_calc_T1R[i] = regress_calc_T1R[i].drop([264,265,266,267,268,269,270,271,272,273,274,275]) #we drop the last 12 observation due to nan values                                                             

In [7]:
"""
Regres forward beta values for the teen portfolios sorted after book equity ratio
"""

regress_calc_BER = {}

for i in BER_val.keys():
    regress_calc_BER[i] = pd.DataFrame()
    regress_calc_BER[i]['ew_forward_beta'] = market_beta_for(df_mkt['EUROPE IM (excess)'], BER_val[i]['ew_excess_ret'],12, 24)['Beta']
    regress_calc_BER[i]['vw_forward_beta'] = market_beta_for(df_mkt['EUROPE IM (excess)'], BER_val[i]['vw_excess_ret'],12, 24)['Beta']                               
    regress_calc_BER[i] = regress_calc_BER[i].drop([264,265,266,267,268,269,270,271,272,273,274,275]) #we drop the last 12 observation due to nan values                                                             

In [8]:
"""
Regres forward beta values for the teen portfolios sorted after Tier 1/risk weighted asset ratio
"""

regress_calc_T1RWA = {}

for i in T1RWA_val.keys():
    regress_calc_T1RWA[i] = pd.DataFrame()
    regress_calc_T1RWA[i]['ew_forward_beta'] = market_beta_for(df_mkt['EUROPE IM (excess)'], T1RWA_val[i]['ew_excess_ret'],12, 24)['Beta']
    regress_calc_T1RWA[i]['vw_forward_beta'] = market_beta_for(df_mkt['EUROPE IM (excess)'], T1RWA_val[i]['vw_excess_ret'],12, 24)['Beta']                               
    regress_calc_T1RWA[i] = regress_calc_T1RWA[i].drop([264,265,266,267,268,269,270,271,272,273,274,275]) #we drop the last 12 observation due to nan values                                                             

## Tabel  - Panel B 

In [9]:
"""
To create a overview of the 3x10x2 portfolios' forward beta and recreate panel B
"""
descriptive21 = pd.DataFrame(columns = ['Equal weighted - Book Equity ratio', 'Value weighted - Book Equity ratio', 'Equal weighted - Tier 1 ratio','Value weighted - Tier 1 ratio', 'Equal weighted - Tier 1/Risk weighted assets ratio', 'Value weighted - Tier 1/Risk weighted assets ratio'])

for i in range(10):
    descriptive21.loc[i+1,'Equal weighted - Book Equity ratio'] = '{:,.02f}'.format(regress_calc_BER[i]['ew_forward_beta'].mean())
    descriptive21.loc[i+1,'Equal weighted - Tier 1 ratio'] = '{:,.02f}'.format(regress_calc_T1R[i]['ew_forward_beta'].mean())
    descriptive21.loc[i+1,'Equal weighted - Tier 1/Risk weighted assets ratio'] = '{:,.02f}'.format(regress_calc_T1RWA[i]['ew_forward_beta'].mean())
    descriptive21.loc[i+1,'Value weighted - Book Equity ratio'] = '{:,.02f}'.format(regress_calc_BER[i]['vw_forward_beta'].mean())
    descriptive21.loc[i+1,'Value weighted - Tier 1 ratio'] = '{:,.02f}'.format(regress_calc_T1R[i]['vw_forward_beta'].mean())
    descriptive21.loc[i+1,'Value weighted - Tier 1/Risk weighted assets ratio'] = '{:,.02f}'.format(regress_calc_T1RWA[i]['vw_forward_beta'].mean())
    
#for last row
descriptive21.loc['10-1','Equal weighted - Book Equity ratio'] = '{:,.02f}'.format(regress_calc_BER[9]['ew_forward_beta'].mean()-regress_calc_BER[1]['ew_forward_beta'].mean())
descriptive21.loc['10-1','Equal weighted - Tier 1 ratio'] = '{:,.02f}'.format(regress_calc_T1R[9]['ew_forward_beta'].mean()-regress_calc_T1R[1]['ew_forward_beta'].mean())
descriptive21.loc['10-1','Equal weighted - Tier 1/Risk weighted assets ratio'] = '{:,.02f}'.format(regress_calc_T1RWA[9]['ew_forward_beta'].mean()-regress_calc_T1RWA[1]['ew_forward_beta'].mean())
descriptive21.loc['10-1','Value weighted - Book Equity ratio'] = '{:,.02f}'.format(regress_calc_BER[9]['vw_forward_beta'].mean()-regress_calc_BER[1]['vw_forward_beta'].mean())
descriptive21.loc['10-1','Value weighted - Tier 1 ratio'] = '{:,.02f}'.format(regress_calc_T1R[9]['vw_forward_beta'].mean()-regress_calc_T1R[1]['vw_forward_beta'].mean())
descriptive21.loc['10-1','Value weighted - Tier 1/Risk weighted assets ratio'] = '{:,.02f}'.format(regress_calc_T1RWA[9]['vw_forward_beta'].mean()-regress_calc_T1RWA[1]['vw_forward_beta'].mean())

In [10]:
descriptive21.reset_index(inplace=True)
descriptive21.rename(columns = {'index':'Decile'})

,Decile,Equal weighted - Book Equity ratio,Value weighted - Book Equity ratio,Equal weighted - Tier 1 ratio,Value weighted - Tier 1 ratio,Equal weighted - Tier 1/Risk weighted assets ratio,Value weighted - Tier 1/Risk weighted assets ratio
0,1,1.14,1.51,1.24,1.52,0.87,1.00
1,2,1.03,1.25,1.17,1.31,1.10,1.24
2,3,0.99,1.34,1.14,1.10,1.13,1.39
3,4,0.92,1.20,1.11,1.22,1.06,1.34
4,5,0.92,1.05,0.93,1.24,0.98,1.35
5,6,0.85,1.17,0.87,1.10,0.88,1.23
6,7,0.80,1.22,0.73,0.94,0.88,1.21
7,8,0.57,0.87,0.59,0.81,0.85,1.35
8,9,0.52,0.94,0.57,0.93,0.69,1.25
9,10,0.60,1.19,0.50,0.93,0.73,1.16


## Piecewise regression - Tier 1 ratio

In [11]:
"""
Saving data in a new dictionary and calculating invers tier 1 ratio in order to do regression
"""

piece_reg = {}
piece_reg = T1R_val.copy()

for i in piece_reg.keys():
    piece_reg[i]['ew_forward_beta'] = regress_calc_T1R[i]['ew_forward_beta'].copy()
    piece_reg[i]['vw_forward_beta'] = regress_calc_T1R[i]['vw_forward_beta'].copy()
    piece_reg[i]['vw_invers_tier1_ratio'] = 1/piece_reg[i]['vw_Tier_1_ratio'] #calculating invers tier 1 ratio
    piece_reg[i]['ew_invers_tier1_ratio'] = 1/piece_reg[i]['ew_Tier_1_ratio'] #calculating invers tier 1 ratio
    piece_reg[i] = piece_reg[i].drop([264,265,266,267,268,269,270,271,272,273,274,275]) #we drop the last 12 observation due to nan values                                                             

In [12]:
"""
Performing piecewise regression on the 10 groups. 
Y = Forward beta
X = Invers Tier 1 ratio 
Both variables are VALUE weighted 
"""

slope_forward = pd.DataFrame()


for i in piece_reg.keys():
    Y = piece_reg[i]['vw_forward_beta'].tolist()
    X = piece_reg[i]['vw_invers_tier1_ratio'].tolist() 
    result = sm.OLS(Y, X).fit()
    slope_forward.loc['Slope',i+1] = '{:,.03f}'.format(result.params[0])
    slope_forward.loc['P-value',i+1] = '{:,.03f}'.format(result.pvalues[0])

slope_forward

,1,2,3,4,5,6,7,8,9,10
Slope,0.039,0.048,0.048,0.059,0.067,0.065,0.062,0.063,0.087,0.129
P-value,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [13]:
"""
Performing piecewise regression on the 10 groups. 
Y = Forward beta
X = Invers Tier 1 ratio 
Both variables are EQUAL weighted 
"""

slope_forward2 = pd.DataFrame()


for i in piece_reg.keys():
    Y = piece_reg[i]['ew_forward_beta'].tolist()
    X = piece_reg[i]['ew_invers_tier1_ratio'].tolist() 
    result = sm.OLS(Y, X).fit()
    slope_forward2.loc['Slope',i+1] = '{:,.03f}'.format(result.params[0])
    slope_forward2.loc['P-value',i+1] = '{:,.03f}'.format(result.pvalues[0])

slope_forward2

,1,2,3,4,5,6,7,8,9,10
Slope,0.028,0.043,0.049,0.053,0.050,0.051,0.049,0.046,0.055,0.069
P-value,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
